# Diabetes
by Kenan Sooklall

Diabetes is a chronic (long-lasting) health condition that affects how your body turns food into energy.

Most of the food you eat is broken down into sugar (also called glucose) and released into your bloodstream. When your blood sugar goes up, it signals your pancreas to release insulin. Insulin acts like a key to let the blood sugar into your body’s cells for use as energy.

Source: [Centers for Disease Control and Preventation](https://www.cdc.gov/diabetes/basics/diabetes.html)

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import shap
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, RepeatedKFold
%matplotlib inline

Dropping
- A lot of missing values
    - weight 
    - payer code
    - medical_specialty
- Only one value
    - examide, citoglipton have 
- Conflict
    - race
- Duplicated patient_nbr

In [ ]:
#df = pd.read_csv('abhishek_diabetes.csv', index_col=[0]).query('Diabetes_012 != "1"').rename(columns={'Diabetes_012': 'label'})
#df['label'] = df['label'].map({0: 0, 2: 1})
drop_cols = ['examide', 'citoglipton', 'encounter_id','weight', 'payer_code', 'medical_specialty', 'race', 'glimepiride-pioglitazone']
df = pd.read_csv('dataset_diabetes/diabetic_data.csv').drop(drop_cols, axis=1).\
            replace('?', np.nan).\
            drop_duplicates(subset=['patient_nbr']).\
            drop(['patient_nbr'], axis=1)
df['readmitted'] = df['readmitted'].map({'NO': 0, '>30': 1, '<30': 1})

In [ ]:
young = 'young'
middle = 'adult'
old = 'old'

age_mapping = {'[0-10)': young, '[10-20)': young, '[20-30)': young, '[30-40)': middle, '[40-50)': middle, '[50-60)': middle,
         '[60-70)': old, '[70-80)': old, '[80-90)': old, '[90-100)': old}
df['age_mapped'] = df['age'].map(age_mapping)

In [ ]:
drug_cols = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
             'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
             'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
             'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 
             'metformin-rosiglitazone', 'metformin-pioglitazone']
diag_cols = ['Circulatory', 'Diabetes', 'Digestive', 'Genitourinary', 'Injury', 
             'Muscoloskeletal', 'Neoplasms', 'Other', 'Respiratory']

In [ ]:
print(df.shape)
df.head()

difference between high blood sugar >200 and very high blood sugar >300
   - how certain medication doses goes up

In [ ]:
def glu_plots(df, target):
    fig, axs = plt.subplots(5, 4, figsize = (15, 10))
    for idx, i in enumerate(axs.flatten()):
        drug = drug_cols[idx]
        sns.countplot(data=df[drug_cols + [target]], x=drug, hue=target, ax=i)#.set(title=drug)
        i.set_yticks([0, 500,1000,1500])
        
        if idx != 3:
            i.legend([],[], frameon=False)
        else:
            i.legend(title=target, loc='upper right')
    
    plt.tight_layout()

In [ ]:
glu_plots(df[df['max_glu_serum'].ne('None')].reset_index(drop=True), target='max_glu_serum')

   - number of changes

In [ ]:
df.columns

The data set contains 22 columns and 253680 rows
- Encounter ID: Unique identifier of an encounter
- Patient number: Unique identifier of a patient
- Race Values: Caucasian, Asian, African American, Hispanic, and other
- Gender Values: male, female, and unknown/invalid
- Age: Grouped in 10-year intervals - 0, 10), 10, 20), …, 90, 100)
- Weight: Weight in pounds
- Admission type: Integer identifier corresponding to 9 distinct values, for example, emergency, urgent, elective, newborn, and not available
- Discharge disposition: Integer identifier corresponding to 29 distinct values, for example, discharged to home, expired, and not available
- Admission source: Integer identifier corresponding to 21 distinct values, for example, physician referral, emergency room, and transfer from a hospital
- Time in hospital: Integer number of days between admission and discharge
- Payer code: Integer identifier corresponding to 23 distinct values, for example, Blue Cross/Blue Shield, Medicare, and self-pay Medical
- Medical specialty: Integer identifier of a specialty of the admitting physician, corresponding to 84 distinct values, for example, cardiology, internal medicine, family/general practice, and surgeon
- Number of lab: procedures Number of lab tests performed during the encounter
- Number of procedures: Numeric Number of procedures (other than lab tests) performed during the encounter
- Number of medications: Number of distinct generic names administered during the encounter
- Number of outpatient: visits Number of outpatient visits of the patient in the year preceding the encounter
- Number of emergency: visits Number of emergency visits of the patient in the year preceding the encounter
- Number of inpatient: visits Number of inpatient visits of the patient in the year preceding the encounter
- Diagnosis 1: The primary diagnosis (coded as first three digits of ICD9); 848 distinct values
- Diagnosis 2: Secondary diagnosis (coded as first three digits of ICD9); 923 distinct values
- Diagnosis 3: Additional secondary diagnosis (coded as first three digits of ICD9); 954 distinct values
- Number of diagnoses: Number of diagnoses entered to the system 0%
- Glucose serum test: result Indicates the range of the result or if the test was not taken. Values: “>200,” “>300,” “normal,” and “none” if not measured
- A1c test result: Indicates the range of the result or if the test was not taken. Values - '>8' if the result was greater than 8%, “>7” if the result was greater than 7% but less than 8%, “normal” if the result was less than 7%, and “none” if not measured. 7% to 8% is an increase of ~30mg/dl of glucose, 6%=120mg/dl
- Change of medications: Indicates if there was a change in diabetic medications (either dosage or generic name). Values - 'change' and 'no change'
- Diabetes medications: Indicates if there was any diabetic medication prescribed. Values: 'yes' and 'no'
- 24 features for medications: glyburide-metformin, glipizide-metformin, glimepiride-pioglitazone, metformin-rosiglitazone, and metformin-pioglitazone, the feature indicates whether the drug was prescribed or there was a change in the dosage. Values - 'up' if the dosage was increased during the encounter, 'down' if the dosage was decreased, 'steady' if the dosage did not change, and 'no' if the drug was not prescribed
- Readmitted Days: Values '<30' if the patient was readmitted in less than 30 days, '>30' if the patient was readmitted in more than 30 days, and 'No' for no record of readmission

source: https://archive-beta.ics.uci.edu/ml/datasets/diabetes+130+us+hospitals+for+years+1999+2008

###  Exploratory Data Analysis

Very lucky to find a dataset with no missing values and no duplicated rows

we can see the data are all on very different scales, which is to be expected. An initial point of concern is  SkinThichkness of 0mm, however that could be more of a percision issue. Another column of concern is BMI, BloodPressure and Glucose, it can't be 0. The min age being greater than 0 is expected and all other values look normal

In [ ]:
metrics = ['mean', 'median', 'std']
df.groupby(['readmitted']).agg({"number_diagnoses": metrics,
                            "num_medications": metrics,
                               'num_procedures': metrics,
                               'time_in_hospital': metrics
                               })

In [ ]:
sns.countplot(x='readmitted', data=df, hue='gender')

gender seems to be mostly independant if someone will be remitted

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(x='readmitted', data=df, hue='age_mapped')
#plt.savefig('age.jpg')

The older portion of the population is the majority for all ranges of readmitted

In [ ]:
df[['age_mapped', 'A1Cresult', 'readmitted']].query('A1Cresult != "None"').sample(50).to_csv('sample_df.csv', index=False)

In [ ]:
plt.figure(figsize=(14, 8))
sns.barplot(x='readmitted', data=df[df['A1Cresult'].ne('None')], hue='age_mapped')

In [ ]:
plt.figure(figsize=(14, 8))
sns.countplot(x='readmitted', data=df[df['max_glu_serum'].ne('None')], hue='max_glu_serum', hue_order=['>300', '>200', 'Norm'])

In [ ]:
sns.scatterplot(x='age', y='time_in_hospital', data=df, hue='readmitted')

## Feature extractor

Quantify categorical data

quantify variables
- age - take the median values
- readmitted - 0: NO 1: other

In [ ]:
df = df.replace(['No','NO', 'None', 'Steady', 'Up', 'Down', 'Yes', 'Ch', 'Norm'], [0,0,0,1,1,1,1,1,1])
df['age'] = df['age'].map(lambda x: sum([int(i) for i in x[1:-1].split('-')]) // 2)
df['max_glu_serum_num'] = df['max_glu_serum'].map({0: 0, 1: 1, '>200': 1, '>300': 1}).astype(int)
df['A1Cresult_num'] = df['A1Cresult'].map({0: 0, 1: 1, '>8': 1, '>7': 1}).astype(int)
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1})
df = df.dropna(subset=['gender'])

Map ids

In [ ]:
def map_diagnosis(df, cols):    
    for i in cols:
        df[i] = pd.to_numeric(df[i], errors='coerce')
    
    for col in cols:
        df["diag"] = 'Other'
        df.loc[df[col].between(459, 785) | df[col].eq(785), "diag"] = "Circulatory"
        df.loc[df[col].between(460, 519) | df[col].eq(786), "diag"] = "Respiratory"
        df.loc[df[col].between(520, 579) | df[col].eq(787), "diag"] = "Digestive"
        df.loc[df[col].between(250, 251), "diag"] = "Diabetes"
        df.loc[df[col].between(800, 999), "diag"] = "Injury"
        df.loc[df[col].between(710, 739), "diag"] = "Muscoloskeletal"
        df.loc[df[col].between(580, 629) | df[col].eq(788), "diag"] = "Genitourinary"
        neoplasms_codes = df[col].between(140, 249) | df[col].between(251, 279) | df[col].isin([780,781,784]) | df[col].between(790, 799)
        df.loc[neoplasms_codes, "diag"] = "Neoplasms"
        df[col] = df["diag"]
        df = df.drop("diag", axis=1)

    return df

In [ ]:
df = map_diagnosis(df, ["diag_1","diag_2","diag_3"])

Get dummies for diag columns

In [ ]:
diag = pd.get_dummies(df['diag_1']) + pd.get_dummies(df['diag_2']) + pd.get_dummies(df['diag_3'])
df = pd.concat([df.drop(df.filter(like='diag_').columns, axis=1), diag], axis=1)

In [ ]:
df = pd.concat([df.drop(['age_mapped'], axis=1), pd.get_dummies(df['age_mapped'])], axis=1)

In [ ]:
plt.figure(figsize=(14, 8))
sns.heatmap(df[drug_cols + ['readmitted']].set_index('readmitted'))

In [ ]:
df.head()

In [ ]:
sns.countplot(data=df[['tolazamide'] + ['readmitted']], x='tolazamide', hue='readmitted')

In [ ]:
fig, axs = plt.subplots(5, 4, figsize = (15, 10))
for idx, i in enumerate(axs.flatten()):
    drug = drug_cols[idx]
    sns.countplot(data=df[drug_cols + ['readmitted']], x=drug, hue='readmitted', ax=i)#.set(title=drug)
    i.set_yticks([0, 1e4,2e4,3e4,4e4])

    if idx != 3:
        i.legend([],[], frameon=False)
    else:
        i.legend(title="Readmitted", loc='upper right')

plt.tight_layout()

In [ ]:
df[df.columns[~df.columns.isin(drug_cols + diag_cols)]].head()

In [ ]:
df['A1Cresult'].value_counts()

Closer look at A1C results and Glucose serum test

In [ ]:
df[df['A1Cresult'].ne(0) & df['max_glu_serum'].ne(0)][glucose_tests + ['readmitted']].

In [ ]:
a1c_df = pd.get_dummies(df[df['A1Cresult'].ne(0)]['A1Cresult']).merge(df[['readmitted']], left_index=True, right_index=True).rename(columns={1: 'normal'}).reset_index(drop=True)
glu_df = pd.get_dummies(df[df['max_glu_serum'].ne(0)]['max_glu_serum']).merge(df[['readmitted']], left_index=True, right_index=True).rename(columns={1: 'normal'}).reset_index(drop=True)
a1c_glu_df = pd.concat([a1c_df, glu_df.drop('readmitted', axis=1)], axis=1).dropna()

In [ ]:
a1c_glu_df['readmitted'].value_counts(normalize=True)

In [ ]:
sns.countplot(x='readmitted', data=df[df['A1Cresult'].ne(0)], hue='A1Cresult')

In [ ]:
sns.countplot(x='readmitted', data=df[df['max_glu_serum'].ne(0)], hue='max_glu_serum')

time in hospital
 - staying longer lead to high readmittion

In [ ]:
df[['time_in_hospital', 'readmitted']].groupby('readmitted')['time_in_hospital'].sum()

Dropped for mostly 0s
 - number_inpatient
 - number_emergency
 - number_outpatient
 - max_glu_serum
 - A1Cresult

In [ ]:
df['readmitted'].value_counts(normalize=True)

# Modeling

In [ ]:
diag_cols

In [ ]:
features = ['gender', 'old', 'young', 'adult', 'time_in_hospital', 'num_lab_procedures', 
            'num_procedures', 'num_medications', 'number_diagnoses', 'diabetesMed', 'change']
glucose_tests = ['max_glu_serum_num', 'A1Cresult_num']
num_cols = ['time_in_hospital','num_lab_procedures', 'num_medications','number_diagnoses']

### Logistic Regression
This is a classification problem so LogisticRegression  is a good starting model

In [ ]:
diab_df = df[features + diag_cols + glucose_tests + ['readmitted']]
#diab_df = a1c_glu_df

In [ ]:
scaler = StandardScaler()
diab_df.loc[:, num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
diab_df.head()

In [ ]:
seed = 42
train_cols = diab_df.drop(['readmitted'], axis=1).columns
X_train, X_test, y_train, y_test = train_test_split(diab_df.drop(['readmitted'], axis=1), diab_df['readmitted'], test_size=0.15)

I fine tune the model using sklearn grid search cross validation on 10 splits. The grid is the search space

In [ ]:
lg_model = LogisticRegression(random_state=seed, penalty='l2').fit(X_train, y_train)
stats = cross_validate(lg_model, X_train, y_train, cv=10, scoring=["accuracy", "precision", "recall", "f1", "roc_auc"])

In [ ]:
y_pred = lg_model.predict(X_test)
logistic_report = classification_report(y_test, y_pred, target_names=['no', 'yes'])
print(logistic_report)

In [ ]:
print('\n'.join(classification_report(y_test, y_pred, target_names=['no', 'yes']).split('\n')[:4]).replace('\n\n', '\n'))

In [ ]:
print(f'Training score: {lg_model.score(X_train, y_train)}\nTesting score: {lg_model.score(X_test, y_test)}')

In [ ]:
y_prob = lg_model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, y_prob)

In [ ]:
feature_imp = pd.DataFrame({'Value': lg_model.coef_[0], 'Feature': X_train.columns})
plt.figure(figsize=(10, 6))
sns.set(font_scale=1)
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[0:10])
plt.title('Features')
plt.tight_layout()
plt.show()

In [ ]:
row_to_show = 6
data_for_prediction = X_test.iloc[row_to_show]  # use 1 row of data here. Could use multiple rows if desired
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

In [ ]:
k_explainer = shap.KernelExplainer(lg_model.predict_proba, X_test)
k_shap_values = k_explainer.shap_values(data_for_prediction)
shap.force_plot(k_explainer.expected_value[1], k_shap_values[1], data_for_prediction)

In [ ]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train,y_train)

In [ ]:
rf_model.score(X_train, y_train)

In [ ]:
rf_model.score(X_test,y_test)

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(sorted(rf_model.feature_importances_),train_cols)

In [ ]:
print(rf_model.predict_proba(data_for_prediction_array))
explainer = shap.TreeExplainer(rf_model)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction)
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction)

In [ ]:
rf_shap_values = explainer.shap_values(X_test)
shap.summary_plot(rf_shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
clf = AdaBoostClassifier(n_estimators=100)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
logistic_report = classification_report(y_test, y_pred, target_names=['no', 'yes'])
print(logistic_report)

In [ ]:
k_explainer = shap.KernelExplainer(clf.predict_proba, X_test)
k_shap_values = k_explainer.shap_values(data_for_prediction)
shap.force_plot(k_explainer.expected_value[1], k_shap_values[1], data_for_prediction)

In [ ]:
gbc = GradientBoostingClassifier(n_estimators=250, learning_rate=.01, max_depth=5, random_state=0)
gbc.fit(X_train, y_train)

y_pred = gbc.predict(X_test)
logistic_report = classification_report(y_test, y_pred, target_names=['no', 'yes'])
print(logistic_report)

### Multi-layer Perceptron

In [ ]:
import torch
import torch.nn as nn

In [ ]:
LEARNING_RATE = 1e-1
DENSE = 64
EPOCHS = 100

In [ ]:
torch.manual_seed(seed)
device = torch.device('cpu')

model = nn.Sequential(
    nn.Linear(X_train.shape[1], DENSE),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(DENSE),
    nn.Linear(DENSE, DENSE//2),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(DENSE//2),
    nn.Linear(DENSE//2, 1)
    )

model.float().to(device)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1, steps_per_epoch=10, epochs=EPOCHS)

In [ ]:
X_tensor = torch.tensor(X_train.values).float().to(device)
y_tensor = torch.tensor(y_train.values.reshape(y_train.shape[0], 1)).float().to(device)

In [ ]:
best_acc = 0
best_model = None
history = []

for i in range(EPOCHS):
    y_pred = model(X_tensor)
    loss = criterion(y_pred.float(), y_tensor)
    acc = ((y_pred > 0.5).int() == y_tensor).sum() / X_tensor.shape[0]
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    history.append({'epoch': i, 'loss': loss.item(), 'acc': acc.item()})
    lr = optimizer.param_groups[0]['lr']
    
    if i % (EPOCHS/10) == 0:
        print('Epoch: {} \tloss: {:.3f} \tacc: {:.3f} \tlr: {:.4f}'.format(i, loss.item(), acc.item(), lr))
        
    if acc > best_acc:
        #print('[+] Saving best')
        best_acc = acc
        best_model = model

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12,8))
train_df = pd.DataFrame(history)
train_df.plot(x='epoch', y='loss', title='Training loss', ax=ax[0])
train_df.plot(x='epoch', y='acc', title='Training Accuracy', ax=ax[1])
plt.tight_layout()

In [ ]:
y_prob = best_model(torch.tensor(X_test.values).float().to(device))
roc_auc_score(y_test, y_prob.detach().numpy())

In [ ]:
y_pred = (y_prob > 0.5).int().detach().numpy()
mlp_report = classification_report(y_test, y_pred, target_names=['Negative', 'Positive'])
print(mlp_report)

### Comparing models

In [ ]:
print('Logistic Regression')
print('-' * 60)
print(logistic_report + '\n')
print('Multi-layer Perceptron')
print('-' * 60)
print(mlp_report)


Over all Logistic regression provided a better model with a higher weighted avg F1-Score and all combinations of precision and recall. Both models suffer from similar problems of low precision and negative examples and  low recall on positive examples. The logistic regression also wins in the fact of how easy it is to train compare to the MLP. The MLP required a lot of tuning and computing power while logistic regression ran quickly.

In [ ]:
Xt_tensor = torch.tensor(X_test.values).float().to(device)

In [ ]:
explainer = shap.DeepExplainer(model, data=X_tensor[:1000])
shap_values = explainer.shap_values(Xt_tensor[:100])

In [ ]:
explainer.expected_value

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0], Xt_tensor)

# Conclusions 

In the United States, 88 million adults, more than 1 in 3 have prediabetes. What’s more, more than 84% of them don’t know they have it. The analysis done in this report is just a tiny fraction of the enormous work put into this field. The models trained here can do better with more data points and more features. However with the limited data the models created performed well.